In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder


In [2]:
import env 
import acquire
import prepare
import model

# ACQUIRE

In [ ]:
raw_df = acquire.get_telco_chunk()

In [ ]:
raw_df.info()

### It's acquired, now
# PREPARE

In [32]:
df = prepare.prep_telco()

In [33]:
# # Use a label encoder to transform the embarked column.
lab_enc = LabelEncoder()

In [34]:
lab_enc.fit(df.online_security)

LabelEncoder()

In [37]:
df.online_security = lab_enc.transform(df.online_security)

In [ ]:
df.online_protection = lab_enc.transform(df.online_protection)
df.device_protection = lab_enc.transform(df.device_protection)
df.tech_support = lab_enc.transform(df.tech_support)
df.streaming_tv = lab_enc.transform(df.streaming_tv)
df.streaming_movies = lab_enc.transform(df.streaming_movies)

In [38]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
customer_id,,,,,,,,,,,,,,,,,,,,
0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,0,Yes,No,Yes,Yes,No,2,Yes,2,65.6,593.30,0
0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,0,No,No,No,No,Yes,1,No,2,59.9,542.40,0
0004-TLHLJ,Male,0,No,No,4,Yes,No,2,0,No,Yes,No,No,No,1,Yes,1,73.9,280.85,1
0011-IGKFF,Male,1,Yes,No,13,Yes,No,2,0,Yes,Yes,No,Yes,Yes,1,Yes,1,98.0,1237.85,1
0013-EXCHZ,Female,1,Yes,No,3,Yes,No,2,0,No,No,Yes,Yes,No,1,Yes,2,83.9,267.40,1


In [22]:
gender = pd.get_dummies(df['gender'], drop_first=True)

In [24]:
online_security = pd.get_dummies(df['online_security'], drop_first=True)

In [9]:
df['gender'] = df['gender'].replace({'Male':1,'Female':0})

In [15]:
df['partner'] = df['partner'].replace({'Yes':1,'No':0})
df['dependents'] = df['dependents'].replace({'Yes':1,'No':0})

## TEST/SPLIT X/y

In [16]:
X = df.drop('churn', axis=1)
#X = df[['monthly_charges']]

In [17]:
y = df['churn']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=123)

In [19]:
X_train.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges
customer_id,,,,,,,,,,,,,,,,,,,
2187-PKZAY,1,0,0,0,12,Yes,No,2,No,No,No,No,No,Yes,1,Yes,3,79.95,1043.40
3402-XRIUO,0,1,1,0,22,Yes,Yes,1,Yes,Yes,No,Yes,No,No,1,Yes,2,63.55,1381.80
9397-TZSHA,0,0,0,0,69,Yes,Yes,3,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,3,No,4,24.60,1678.05
9153-BTBVV,0,0,1,0,71,Yes,Yes,3,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,3,No,3,25.00,1753.00
3793-MMFUH,0,1,0,0,13,Yes,Yes,2,No,No,No,No,Yes,Yes,1,Yes,1,95.05,1290.00


### PREP is boring, Let's 
# MODEL AND CHART STUFF. 
# OR EXPLORE. 
I guess.

In [ ]:
sns.countplot(x='churn', data=df)

In [ ]:
sns.countplot(x='churn', data=df, hue='dependents')

In [ ]:
#Boxplot to show the distribution of ages. X is class, Y is age.
plt.figure(figsize = (10,7))
sns.boxplot(x='churn', y='tenure', data=df)

# MODEL, Decision-Tree

In [14]:
model.do_the_decisionTree('Entropy', X_train, y_train)

ValueError: could not convert string to float: 'No'

# MODEL, log-reg

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()

In [ ]:
logmodel.fit(X_train, y_train)

In [ ]:
predictions = logmodel.predict(X_train)
predictions_probas = logmodel.predict_proba(X_train)

In [ ]:
predictions_probas[:,0] < .5

In [ ]:
from sklearn import metrics

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_train, predictions)

In [ ]:
cnf_matrix

In [ ]:
#Looks like using logistic regression with just monthly charges as the predictor is worthless and predicts nobody
#will churn